In [14]:
import os 
import getpass
import random

import pickle
import numpy as np
import pandas as pd
from scipy import stats

from sklearn import cross_validation, preprocessing
from sklearn.externals import joblib
from sklearn import pipeline
from sklearn import linear_model
from sklearn import ensemble
from sklearn import metrics

%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


### Load data

#### Kaggle based data

https://www.kaggle.com/petrkajzar/wikiskripta-globalstats/downloads/WikiSkripta.csv/2

In [96]:
data = pd.read_csv('/data/sazulay/test_out/WikiSkripta.csv')
del data['date']
data.head()

,total,good,views,edits,users,admins,images,activeusers,valid_checked,checked
0,697,198,121769,4383,143,6,113,NaN,NaN,NaN
1,707,203,122457,4412,143,6,115,NaN,NaN,NaN
2,729,203,123673,4468,143,6,133,NaN,NaN,NaN
3,789,205,125181,4590,143,6,188,NaN,NaN,NaN
4,791,205,126063,4596,146,6,188,NaN,NaN,NaN


In [97]:
data.describe()

,total,good,views,edits,users,admins,images,activeusers,valid_checked,checked
count,3359.000000,3359.000000,3.359000e+03,3359.000000,3359.000000,3359.000000,3359.000000,2707.000000,2561.000000,2561.000000
mean,41098.216433,6389.111938,9.317807e+07,234300.493302,8096.426615,48.239059,3674.381364,89.762837,484.208122,847.558376
std,21214.288790,2705.640776,7.986733e+07,126742.380277,5279.054449,26.210623,1735.428287,48.784016,193.288495,126.028995
min,697.000000,198.000000,1.217690e+05,4383.000000,143.000000,6.000000,113.000000,27.000000,74.000000,489.000000
25%,24065.500000,4718.500000,1.301702e+07,142115.000000,3270.500000,22.000000,1987.000000,49.000000,265.000000,788.000000
50%,45437.000000,7302.000000,7.607001e+07,247286.000000,8379.000000,52.000000,4342.000000,74.000000,582.000000,886.000000
75%,58283.500000,8599.500000,1.682811e+08,347509.000000,11030.500000,67.000000,5120.500000,116.000000,640.000000,918.000000
max,69510.000000,9473.000000,2.383516e+08,413487.000000,16252.000000,90.000000,5812.000000,259.000000,763.000000,1100.000000


#### Mean impute data

In [98]:
imp = preprocessing.imputation.Imputer()

In [99]:
imp.fit(data)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)

In [100]:
data_tr = pd.DataFrame(imp.transform(data), columns=data.columns)

In [101]:
data.corr()

,total,good,views,edits,users,admins,images,activeusers,valid_checked,checked
total,1.000000,0.985984,0.931635,0.997832,0.973633,0.983278,0.983553,-0.411842,-0.670468,0.939049
good,0.985984,1.000000,0.863017,0.979969,0.924541,0.942922,0.972625,-0.397565,-0.599263,0.964899
views,0.931635,0.863017,1.000000,0.942697,0.976244,0.975131,0.915717,-0.438463,-0.701915,0.911537
edits,0.997832,0.979969,0.942697,1.000000,0.974565,0.985414,0.976092,-0.413038,-0.671794,0.931735
users,0.973633,0.924541,0.976244,0.974565,1.000000,0.992308,0.951385,-0.423358,-0.743031,0.884775
admins,0.983278,0.942922,0.975131,0.985414,0.992308,1.000000,0.967750,-0.424996,-0.686979,0.924070
images,0.983553,0.972625,0.915717,0.976092,0.951385,0.967750,1.000000,-0.392061,-0.516659,0.967331
activeusers,-0.411842,-0.397565,-0.438463,-0.413038,-0.423358,-0.424996,-0.392061,1.000000,0.382949,-0.362558
valid_checked,-0.670468,-0.599263,-0.701915,-0.671794,-0.743031,-0.686979,-0.516659,0.382949,1.000000,-0.455482
checked,0.939049,0.964899,0.911537,0.931735,0.884775,0.924070,0.967331,-0.362558,-0.455482,1.000000


### Build linear model to predict #active_users

In [102]:
y = data_tr['activeusers']
del data_tr['activeusers']
X = data_tr

In [103]:
lin_pred = linear_model.LinearRegression()
lin_pred.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [104]:
lin_pred.score(X, y)

0.24627153781590339

In [105]:
coefs = pd.DataFrame(
    pred.coef_.reshape(1, -1),
    columns=data_tr.columns)
coefs.transpose().sort(columns=0, ascending=False).head()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,0
checked,0.106197
valid_checked,0.025737
users,0.007222
images,0.006454
edits,0.002025


##  t (Student's) distribiution

--------------------------------------
**Let's say X was sampled from a normal distribution:**
X  ~ ${N(μ, σ2)}$

The mean of the samples is:
\begin{equation*}
\bar{X} = \frac{1}{{n}} \sum_{i=1}^n X_i
\end{equation*}

And the estimated variance:
\begin{equation*}
S_n = \frac{1}{{n - 1}} \sum_{i=1}^n (X_i - \bar{X})^2
\end{equation*}

Then the following statistics:
\begin{equation*}
Z = \frac{\bar{X} - \mu}{{\sigma / \sqrt{n}}}
\end{equation*}
is ~N(μ, σ2)

And the following statistics:
\begin{equation*}
t = \frac{\bar{X} - \mu}{{S_n / \sqrt{n}}}
\end{equation*}
has a Student's t-distribution with n − 1 degrees of freedom ~ $t(df=n-1)$

------------------
**t-ditribution can be used to find the propbabilty that X was sampled from mean μ**

if we guess that $\hat{\mu}$ is a good representation of the real ${\mu}$

then:

$t_0 = \frac{\bar{X} - \hat{\mu}}{{S_n / \sqrt{n}}}$ ~ $t_{n-1}$
--

let's choose 95% certainty, let T be the value where $P(t_{n-1} < T) = 0.95$

then if:

$|{t_0}| > T$

we can reject the null hypothesis $H_0: \hat{\mu} = {\mu}$


------------------
**t-ditribution can be used to find the confidence intervals for X**

$t = \frac{\bar{X} - \mu}{{Sn / \sqrt{n}}}$ ~ $t_{n-1}$

$P(t_{n-1} < T) = 0.95 \longleftrightarrow T := t_{.95, n-1}$

Then $X = \mu \pm T * \frac{Sn}{{\sqrt{n}}}$

where : $se(X) = \frac{Sn}{{\sqrt{n}}}$, is the estimated standard error of X


In [106]:
def t_stat(perc, df):
    alpha = 1 - perc
    return stats.t.ppf(1 - alpha/2, df=df)

In [107]:
t_stat(.95, df=100)

1.9839715184496334

In [188]:
def t_test(t_val, df):
    return (1 - stats.t.cdf(abs(t_val), df=df)) * 2

In [192]:
t_test(1.983, df=100)

0.05011015368624272

## Simple linear regresson

--------------------------
$y_i = \beta_0 + \beta_1 * X_i + \epsilon$

Base assumption is always that $\epsilon \tilde{} N(0, \sigma^2)$

Therefore, $yi \tilde{} N(\beta_0 + \beta_1 * X_i, \sigma^2)$

--------------------------
It can be shown that, where:

$S_{xx} = \sum_{i=1}^n{(X_i - \bar{X})}^2$

$S_{xy} = \sum_{i=1}^n{y_i*(X_i - \bar{X})}$


--------------------------
Then: 

$\hat{\beta_1} = \frac{S_{xx}}{{S_{xy}}}$

$\hat{\beta_0} = \bar{y} - \hat{\beta_1}*\bar{X}$

$\hat{\beta_1} \tilde{} N(\beta_1, \frac{\sigma^2}{{S_{xx}}})$
--

---------------
Therefore:

$Z_1 = \frac{\hat{\beta_1} - \beta_1}{{\frac{\sigma}{\sqrt{{S_{xx}}}}}} \tilde{} N(0, 1)$
--

$t_1 = \frac{\hat{\beta_1} - \beta_1}{{\frac{\hat{\sigma}}{\sqrt{{S_{xx}}}}}} \tilde{} t_{n-2}$
--

where $\hat{\sigma}^2 = se(\epsilon)^2 = \frac{SS_{res}}{{n-2}}$

where $SS_{res} = \sum_{i=1}^n{e_i^2} = \sum_{i=1}^n{(y_i - \hat{y}_i)^2}$

### Rejecting the Null Hypothesis

Let's first test the null hyphotesis $H_0: \beta_1 = 0$

Meaning - there is no linear connection between the X and y

$t_1 =  \frac{\hat{\beta_1} - 0}{{\frac{\hat{\sigma}}{\sqrt{{S_{xx}}}}}}$
--

Example of a possible result (${\beta_1} = 0.265, {\frac{\hat{\sigma}}{\sqrt{{S_{xx}}}}} = 0.0026$):

$t_1 = \frac{0.265 - 0}{{0.0026}} = 102.18 \gg t_{.95, n-2} = 1.96$
--

we can easily reject $H_0$

## Multiple Linear Regresssion Case

$y_i = \beta_0 + \sum_{j=1}^k{\beta_jX_i} + \epsilon_i$

$p = k + 1$, is the ammount of variables

$\vec{y} = \textbf{X}\cdot\vec{\beta}$

$LS(\beta) = \sum_{i=1}^n{\epsilon_i} = \vec{\epsilon}'\cdot\vec{\epsilon} = (y - \textbf{X}\cdot\vec{\beta})'\cdot(y - \textbf{X}\cdot\vec{\beta})$

$\frac{\partial{LS(\beta)}}{{\partial{\beta}}} = -2\textbf{X'}y + 2\textbf{X'X}\beta$

$\Downarrow$

$\hat{\beta} = \textbf{(X'X)}^{-1}\textbf{X'}y$

$\hat{y} = \textbf{X}\hat{\beta} = \textbf{X}\textbf{(X'X)}^{-1}\textbf{X'}y := \textbf{H}y$

-----------------------------
Again, base assumption is always that $\epsilon \tilde{} N(0, \sigma^2)$

**Then it can be shown that** $E(\hat{\beta}) = \beta$

$Var(\hat{\beta}) = Var(\textbf{(X'X)}^{-1}\textbf{X'}y) = \textbf{(X'X)}^{-1}\textbf{X'}\cdot Var(y) \cdot[\textbf{(X'X)}^{-1}\textbf{X'}]' = \sigma^2\cdot\textbf{(X'X)}^{-1}$ 

So before we saw: $\hat{\beta_1} \tilde{} N(\beta_1, \frac{\sigma^2}{{S_{xx}}})$

And now: $\hat{\beta} \tilde{} N(\beta, \sigma^2\cdot\textbf{(X'X)}^{-1})$


-----------
Again, the estimation of the real $\sigma$:

$\hat{\sigma}^2 = se(\epsilon)^2 = \frac{SS_{res}}{{n-p}}$

where $SS_{res} = \sum_{i=1}^n{e_i^2} = \sum_{i=1}^n{(y_i - \hat{y}_i)^2} = (y - \textbf{X}\hat{\beta})'(y - \textbf{X}\hat{\beta}) = yy' - \hat{\beta}'\textbf{X}'y$

### Computing beta by definition

In [108]:
XtX = X.transpose().dot(X)
XtX_inv = pd.DataFrame(np.linalg.pinv(XtX.values), XtX.columns, XtX.index)

In [110]:
XtX.shape, X.shape

((9, 9), (3359, 9))

In [111]:
XtX.head()

,total,good,views,edits,users,admins,images,valid_checked,checked
total,7.184819e+12,1.072052e+12,1.816372e+16,4.135417e+13,1.483854e+12,8.495307e+09,6.288387e+11,6.258201e+10,1.208970e+11
good,1.072052e+12,1.616991e+11,2.625935e+15,6.156787e+12,2.181015e+11,1.259805e+09,9.419167e+10,1.001833e+10,1.858136e+10
views,1.816372e+16,2.625935e+15,5.058333e+19,1.053763e+17,3.916241e+15,2.195284e+13,1.576231e+15,1.268960e+14,2.861488e+14
edits,4.135417e+13,6.156787e+12,1.053763e+17,2.383398e+14,8.561636e+12,4.895743e+10,3.612736e+12,3.548334e+11,6.907760e+11
users,1.483854e+12,2.181015e+11,3.916241e+15,8.561636e+12,3.137717e+11,1.772968e+09,1.291966e+11,1.164974e+10,2.422927e+10


In [112]:
beta_hat = XtX_inv.dot(X.transpose()).dot(y)

In [115]:
pd.DataFrame(beta_hat).sort(columns=0, ascending=False).head()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,0
checked,0.109331
valid_checked,0.023983
users,0.006660
images,0.006296
edits,0.002064


Indeed identical to what we saw from sklearn lin reg

### Estimate $\hat{\sigma}^2$

Reminder:

$\hat{\sigma}^2 = se(\epsilon)^2 = \frac{SS_{res}}{{n-p}}$

where $SS_{res} = \sum_{i=1}^n{e_i^2} = \sum_{i=1}^n{(y_i - \hat{y}_i)^2} = (y - \textbf{X}\hat{\beta})'(y - \textbf{X}\hat{\beta}) = yy' - \hat{\beta}'\textbf{X}'y$

-----

In [119]:
n = X.shape[0]  # num of samples
k = X.shape[1]  # num of columns (features)
p = k + 1 
k, p

(9, 10)

In [120]:
SSres = y.dot(y.transpose()) - beta_hat.transpose().dot(X.transpose()).dot(y)
SSres

4857312.3821723461

In [121]:
sigma_hat = np.sqrt(SSres / (n - p))
sigma_hat

38.083814712020541

### Coef Confidence Intervals

Reminder:

$\hat{\beta} \tilde{} N(\beta, \sigma^2\cdot\textbf{(X'X)}^{-1})$

Where $C_{ij}$ is the covariance matrix: $C_{ij} = {\textbf{(X'X)}^{-1}}_{ij}$

Meaning each coef $\hat{\beta_i} \tilde{} N(\beta_i, \sigma^2\cdot C_{ii})$

$\Downarrow$

$t_i = \frac{\hat{\beta_i} - \beta_i}{{\hat{\sigma}}{\sqrt{C_{ii}}}} \tilde{} t_{n-p}$
---


$\Downarrow$

$\hat{\beta_i} = \hat{\beta_i} \pm t_{.95, n-p}\cdot\hat{\sigma}\cdot\sqrt{C_{ii}}$
---


In [150]:
C = XtX_inv

sqrtCii = pd.DataFrame(np.diag(np.sqrt(C)), index=[C.columns], columns=['sqrtCii'])
sqrtCii

,sqrtCii
total,4.872997e-05
good,1.530905e-04
views,3.395161e-09
edits,5.960729e-06
users,5.548149e-05
admins,5.920767e-07
images,1.171170e-04
valid_checked,1.995334e-04
checked,1.476976e-04


In [157]:
t = (beta_hat - 0).divide(sigma_hat * sqrtCii['sqrtCii'])
t

total            -2.116973
good             -6.507829
views           -14.227834
edits             9.094071
users             3.151778
admins           27.033301
images            1.411471
valid_checked     3.156065
checked          19.436978
dtype: float64

In [193]:
p_values = t.apply(lambda t_val: t_test(t_val, df=(n - p)))
p_values

total            3.433566e-02
good             8.761458e-11
views            0.000000e+00
edits            0.000000e+00
users            1.637129e-03
admins           0.000000e+00
images           1.581987e-01
valid_checked    1.613325e-03
checked          0.000000e+00
dtype: float64